In [1]:
import bitarray

def print_hex_bytes(src):
    for i in range(16):
        str = src[i*16:(i+1)*16]
        print(' '.join('{:02X}'.format(a) for a in str))

In [2]:
rate = 1088

In [3]:
def take_by_ijk(x,y,z):
    return 64*(5*y + x) + z

In [4]:
def pad10x1(x, m):   
    j = (-m-2)%x
    if j < 0:
        j += x
    additional = bitarray.bitarray(j + 2, endian="little")
    additional.setall(False)
    additional[0] = True
    additional[j+1] = True
    return additional

In [5]:
def Theta(S):
    new_S = bitarray.bitarray(endian="little")
    new_S = S.copy()
    new_S.setall(False)
    C = bitarray.bitarray(endian="little")
    C = S.copy()
    C.setall(False)
    D = bitarray.bitarray(endian="little")
    D = S.copy()
    D.setall(False)

    # S[take_by_ijk(x,y,z)] == A[x,y,z]

    for x in range(5):
        for z in range(64):
            C[take_by_ijk(x,0,z)] = S[take_by_ijk(x,0,z)] ^ S[take_by_ijk(x,1,z)] ^ S[take_by_ijk(x,2,z)] ^ S[take_by_ijk(x,3,z)] ^ S[take_by_ijk(x,4,z)]

    for x in range(5):
        for z in range(64):
            D[take_by_ijk(x,0,z)] = C[take_by_ijk((x-1) % 5,0,z)] ^ C[take_by_ijk((x+1) % 5,0,(z-1) % 64)]
            for y in range(5):
                new_S[take_by_ijk(x,y,z)] = S[take_by_ijk(x,y,z)] ^ D[take_by_ijk(x,0,z)]

    return new_S        


In [6]:
def Rho(S):
    new_S = S.copy()
    new_S.setall(False)
    # for k in range(64):
    #     new_S[take_by_ijk(0,0,k)] = S[take_by_ijk(0,0,k)]
    new_S[:65] = S[:65]

    i, j = 1, 0
    for t in range(24):
        for k in range(64):
            idx = (t+1)*(t+2)//2
            new_S[take_by_ijk(i,j,k)] = S[take_by_ijk(i,j, ((k-idx) % 64))]
        i,j = j, (2*i + 3*j) % 5
         
    return new_S
        

In [7]:
def Pi(S):
    new_S = S.copy()
    new_S.setall(False)
    for i in range(5):
        for j in range(5):
            for k in range(64):
                new_S[take_by_ijk(i,j,k)] = S[take_by_ijk((i + 3*j) % 5, i, k)]
    return new_S

In [8]:
def Chi(S):
    new_S = S.copy()
    new_S.setall(False)
    for i in range(5):
        for j in range(5):
            for k in range(64):
                new_S[take_by_ijk(i,j,k)] = S[take_by_ijk(i,j,k)] ^ ((S[take_by_ijk((i+1)%5,j,k)] ^ 1) * S[take_by_ijk((i+2)%5, j, k)])
    return new_S

In [9]:
def rc(t):
    if t % 255 == 0:
        return 1
    R = bitarray.bitarray(8, endian="little")
    R.setall(False)
    R[0] = True
    for i in range(1, t % 255 + 1):
        R.insert(0, False)
        R[0] = R[0] ^ R[8]
        R[4] = R[4] ^ R[8]
        R[5] = R[5] ^ R[8]
        R[6] = R[6] ^ R[8]
        R = R[0:8]
        
    return R[0]

In [10]:
def Iota(S, Ir):
    new_S = S.copy()
    array_rc = bitarray.bitarray(64, endian="little")
    array_rc.setall(False)
    for j in range(6+1):
        array_rc[(2 ** j) - 1] = rc(j + 7 * Ir)
    
    for k in range(64):
        old_digit = new_S[take_by_ijk(0,0,k)]
        new_S[take_by_ijk(0,0,k)] = old_digit ^ array_rc[k]
    
    return new_S

In [11]:
def Rnd(S, Ir):
    res_Theta = Theta(S)
    res_Rho = Rho(res_Theta)
    res_Pi = Pi(res_Rho)
    res_interim = Chi(res_Pi)
    res = Iota(res_interim, Ir)
    return res

In [12]:
def Keccak_p(S):
    new_S = bitarray.bitarray(endian="little")
    new_S = S.copy()
    for ir in range(24):  
        new_S = Rnd(new_S, ir)
    return new_S

In [13]:
def Keccak_f(b):
    return Keccak_p(b)

In [14]:
def Sponge(N, d):
    P = bitarray.bitarray(endian="little")
    P = N.copy()
    extended = pad10x1(rate, len(N))
    P.extend(extended)
    n = len(P)/rate
    c = d - rate
    zeros_S = bitarray.bitarray(d, endian="little")
    zeros_S.setall(False)
    
    if len(P) % rate == 0:
        n = int(n)
    else:
        print("IN SPONGE: ", len(P), " / ", rate, ' = ', n)
        
    for i in range(n): 
        Pi = P[i * rate : (i+1) * rate]
        c_zeros = bitarray.bitarray(c, endian="little")
        c_zeros.setall(False)
        Pi.extend(c_zeros)
        spacing = zeros_S ^ Pi
        zeros_S = Keccak_f(spacing)

    return zeros_S[0:256]


In [15]:
def Keccak(N,d):
    N.extend([0,1])
    res = Sponge(N,d)
    return res

In [16]:
def SHA3_256(M,d):
    return Keccak(M,d)

In [17]:
file = open("test.txt", mode="rb")
source_bitArray = bitarray.bitarray(endian="little")
source_bitArray.fromfile(file)
file.close()
msg30 = bitarray.bitarray(endian="little")
msg30.extend([1,1,0,0,1,0,1,0,0,0,0,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,0,1,1,0])
msg5 = bitarray.bitarray(endian="little")
msg5.extend([1,1,0,0,1])
msg0 = bitarray.bitarray(endian="little")


In [18]:
res = bitarray.bitarray()
res = SHA3_256(msg0, 1600)
print_hex_bytes(res.tobytes())
result_file = open("sha3_hash.txt", mode="wb")
result_file.write(res)
result_file.close()

A7 FF C6 F8 BF 1E D7 66 51 C1 47 56 A0 61 D6 62
F5 80 FF 4D E4 3B 49 FA 82 D8 0A 4B 80 F8 43 4A














